# Sherlock

In [ ]:
#!wget https://www.gutenberg.org/files/1661/1661-0.txt > sherlock.txt

In [ ]:
!pip install -qU sentence-transformers bioinfokit

In [ ]:
from nltk import sent_tokenize, word_tokenize
import nltk

nltk.download('punkt')

In [ ]:
f = open('sherlock.txt', 'r')
contents = f.read().replace('\n', ' ')
print(contents[:2000])

In [ ]:
sent_text = sent_tokenize(contents)
print(sent_text[:30])

# for sentence in sent_text:
#     tokenized_text = nltk.word_tokenize(sent_text)
# tagged = nltk.pos_tag(tokenized_text)
# print(tagged)
# print(tokenized_text[:40])

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings = model.encode(sent_text)

In [ ]:
print(sentence_embeddings.shape)

In [ ]:
from sklearn.manifold import TSNE
tsne_emb = TSNE(n_components=2, perplexity=30.0, n_iter=1000, verbose=1).fit_transform(sentence_embeddings)

In [ ]:
# https://www.reneshbedre.com/blog/tsne.html

# from bioinfokit.visuz import cluster
# cluster.tsneplot(score=tsne_emb, dotsize=1)
# # color_class = sentence_y
# # cluster.tsneplot(score=fashion_tsne, colorlist=color_class, legendpos='upper right', legendanchor=(1.15, 1) )

In [ ]:
# https://scipy-lectures.org/packages/scikit-learn/auto_examples/plot_tsne.html
# Alternative visualization using SKLearn, in color, clustered by every 500 sentences

from matplotlib import pyplot as plt
plt.figure(dpi=200)
colors = 'r', 'g', 'b', 'cyan', 'm', 'y', 'k', 'gray', 'teal', 'orange', 'purple'
for c_idx, n in enumerate(range(0,4500,500)):
    sents = tsne_emb[n:n+500]
    color = colors[c_idx]
    l = "{}-{}".format(n, n+500)
    plt.scatter(*zip(*sents), c=color, label=l, s=2)
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('tsne_color.png')
plt.show()

In [ ]:
from IPython.display import Image
Image(filename='tsne_2d.png') 